In [548]:
import pandas as pd
from src import *
import matplotlib.pyplot as plt
from string import punctuation
import nltk
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer, RegexpTokenizer, TweetTokenizer
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression , SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split , cross_validate , GridSearchCV, cross_val_predict
from sklearn.pipeline import make_pipeline , Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, recall_score, f1_score
from nltk.corpus import stopwords as sw
from sklearn.ensemble import RandomForestClassifier
from nltk.tokenize import  word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
pd.set_option ('display.max_colwidth', None)
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [549]:
def mean_scores(results):
    '''Return Train/Test Mean Score from a Cross Val'''
    return print(f'''Mean Train Score: {results['train_score'].mean()}
Mean Test Score: {results['test_score'].mean()}''')

def metrics(y_true,y_preds):
    '''Gives recall,precision,accuracy,F1 score and confusion matrix'''
    return print(f'''Recall Score: {recall_score(y_true,y_preds,average='micro')}
Precision Score: {precision_score(y_true,y_preds,average='micro')}
Accuracy Score:{accuracy_score(y_true,y_preds)}
F1 Score: {f1_score(y_true,y_preds,average='micro')}
Confusion Matrix: 
{confusion_matrix(y_true,y_preds)}''')

def gs_mean_scores(results):
    '''Return Train and Test Scores from a grid search'''
    return print(f'''Mean Train Score: {results.cv_results_['mean_train_score'].mean()}'
Mean Test Score: {results.cv_results_['mean_test_score'].mean()}''')

def company_search(test):
    if test == 'iphone':
        return 'apple'
    elif test == 'apple':
        return 'apple'
    elif test == 'ipad':
        return 'apple'
    elif test == 'ipad or iphone app':
        return 'apple'
    elif test == 'itunes':
        return 'apple'
    elif test == 'other apple product or service':
        return 'apple'
    elif test == 'ios':
        return 'apple'
    elif test == 'ipadquot':
        return 'apple'
    elif test == 'applequot':
        return 'apple'
    elif test == 'iphonequot':
        return 'apple'
    elif test == 'quotiphone':
        return 'apple'
    elif test == 'iosquot':
        return 'apple'
    elif test =='quotipad':
        return 'apple'
    elif test == 'quotapple':
        return 'apple'
    elif test =='google':
        return 'google'
    elif test == 'android':
        return 'google'
    elif test == 'other google product or service':
        return 'google'
    elif test == 'android app':
        return 'google'
    elif test == 'quotgoogle':
        return 'google'
    elif test == 'googlequot':
        return 'google'
    elif test == 'androidquot':
        return 'google'
    else:
        return 'unknown'

In [550]:
df = pd.read_csv('data/judge-1377884607_tweet_product_company.csv',encoding='latin1')
df.isna().sum()

tweet_text                                               1
emotion_in_tweet_is_directed_at                       5802
is_there_an_emotion_directed_at_a_brand_or_product       0
dtype: int64

There are 5802 missing values telling what the tweet is directed at. A way to combat this would be to look at the tweet and look for keywords such as iphone or ipad and assign who it is directed to that way.

In [551]:
df = df.dropna(subset=['tweet_text'])

In [553]:
tweets = TweetTokenizer(preserve_case=False,reduce_len=True)
tweeted = df['tweet_text'].apply(tweets.tokenize)
tweeted.str.join(' ')

0                     . @wesley83 i have a 3g iphone . after 3 hrs tweeting at #rise_austin , it was dead ! i need to upgrade . plugin stations at #sxsw .
1          @jessedee know about @fludapp ? awesome ipad / iphone app that you'll likely appreciate for its design . also , they're giving free ts at #sxsw
2                                                                        @swonderlin can not wait for #ipad 2 also . they should sale them down at #sxsw .
3                                                                      @sxsw i hope this year's festival isn't as crashy as this year's iphone app . #sxsw
4                @sxtxstate great stuff on fri #sxsw : marissa mayer ( google ) , tim o'reilly ( tech books / conferences ) & matt mullenweg ( wordpress )
                                                                               ...                                                                        
9088                                                                  

In [554]:
tweeted = tweeted.str.replace(r'[^\w\s]', '')

In [555]:
tweeted

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
9088   NaN
9089   NaN
9090   NaN
9091   NaN
9092   NaN
Name: tweet_text, Length: 9092, dtype: float64

In [412]:
#Take out puncuation and other unnecessary characters
df['tweet_text'] = df['tweet_text'].str.replace(r'[^\w\s]', '')
df['tweet_text'] = df['tweet_text'].str.replace('quot', '')
# Make lower case and split
df['tweet_text'] = df['tweet_text'].str.lower()
df['tweet_text'] = df['tweet_text'].str.split(' ')
# Drop the 1 NA tweet we have
df = df.dropna(subset=['tweet_text'])

Here we do some basic cleaning of the data set, removing puncuation making everything lower case and splitting the data.

In [413]:
# Make a blank column to set up for some data cleaning
df['test'] = ''
# Fill in the values with a string na to avoid errors
df.fillna('na',inplace=True)


In [414]:
# This salvages alot of our data

apple = ['iphone','ipad','apple','apples','ipads']
android = ['android','google','androids','googles']

# Loop through the tweet column and look for keywords to assign product to
for ind , val in enumerate(df['tweet_text']):
    if df['emotion_in_tweet_is_directed_at'].values[ind] == 'na':
        for y in val:
            if y in apple:
                df['test'].values[ind] = y
            elif y in android:
                df['test'].values[ind] = y
                
    else:
        df['test'].values[ind] = df['emotion_in_tweet_is_directed_at'].values[ind]
      

Since there were such a high amount of NA values in the emotion tweeted at column, we thought it was best to try to salvage as much data as possible. We looked for keywords in the tweets and assigned a product to them based on what they 

In [415]:
#make everything lower case to make values match
df['test'] = df['test'].map(lambda x : x.lower())
df['test'].unique()

array(['iphone', 'ipad or iphone app', 'ipad', 'google', 'android',
       'apple', 'android app', 'other google product or service', '',
       'googles', 'ipads', 'apples', 'other apple product or service',
       'androids'], dtype=object)

In [416]:
#This cell replaces words with the rightful product to reduce unique values
df['test'] = df['test'].replace({'googles': 'google', 'apples': 'apple', 'androids': 'android','ipads':'ipad'})

In [417]:
df['test'].unique()

array(['iphone', 'ipad or iphone app', 'ipad', 'google', 'android',
       'apple', 'android app', 'other google product or service', '',
       'other apple product or service'], dtype=object)

We went from 5802 NA values to almost under 900! The remainder tweets do not mention anything about either product, and therefore are not useful for our analysis.

In [418]:
df['is_there_an_emotion_directed_at_a_brand_or_product'].unique()

array(['Negative emotion', 'Positive emotion',
       'No emotion toward brand or product', "I can't tell"], dtype=object)

Looking at the target of our dataset, there seems to be four different values. Negative emotion, positive emotion, no emotion and I can't tell. Let's get a closer look at these 'I can't tell' labels.

In [419]:
df.loc[df['is_there_an_emotion_directed_at_a_brand_or_product'] == "I can't tell"]

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,test
90,"[thanks, to, mention, for, publishing, the, news, of, mention, new, medical, apps, at, the, sxswi, conf, blog, link, sxsw, sxswh]",na,I can't tell,
102,"[ûïmention, apple, has, opened, a, popup, store, in, austin, so, the, nerds, in, town, for, sxsw, can, get, their, new, ipads, link, wow]",na,I can't tell,ipad
237,"[just, what, america, needs, rt, mention, google, to, launch, major, new, social, network, called, circles, possibly, today, link, sxsw]",na,I can't tell,google
341,"[the, queue, at, the, apple, store, in, austin, is, four, blocks, long, crazy, stuff, sxsw]",na,I can't tell,apple
368,"[hope, its, better, than, wave, rt, mention, buzz, is, googles, previewing, a, social, networking, platform, at, sxsw, link]",na,I can't tell,google
...,...,...,...,...
9020,"[its, funny, watching, a, room, full, of, people, hold, their, ipad, in, the, air, to, take, a, photo, like, a, room, full, of, tablets, staring, you, down, sxsw]",na,I can't tell,ipad
9032,"[mention, yeah, we, have, mention, , google, has, nothing, on, us, , sxsw]",na,I can't tell,google
9037,"[mention, yes, the, google, presentation, was, not, exactly, what, i, was, expecting, sxsw]",na,I can't tell,google
9058,"[do, you, know, what, apple, is, really, good, at, making, you, feel, bad, about, your, xmas, present, , seth, meyers, on, ipad2, sxsw, doyoureallyneedthat]",na,I can't tell,apple


Since this makes up such a small amount of our data, we will move the I can't tell value to that of neutral, along with the value 'No emotion toward brand or product'.

In [420]:
# Here we make change targets into more readable text
emotion_dict = {'No emotion toward brand or product' : 'Neutral emotion', "I can't tell": 'Neutral emotion', 'Positive emotion': 'Positive emotion',
               'Negative emotion': 'Negative emotion'}
df['is_there_an_emotion_directed_at_a_brand_or_product'] = df['is_there_an_emotion_directed_at_a_brand_or_product'].map(emotion_dict)


In [421]:
df['is_there_an_emotion_directed_at_a_brand_or_product'].unique()

array(['Negative emotion', 'Positive emotion', 'Neutral emotion'],
      dtype=object)

In [422]:
df['product'] = df['test'].map(company_search)

In [423]:
df.loc[df['product'] == 'unknown']

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,test,product
51,"[ûïmention, link, lt, help, me, forward, this, doc, to, all, anonymous, accounts, techiesamp, ppl, who, can, help, us, jam, libya, sxsw]",na,Neutral emotion,,unknown
52,"[¼, what, _, link, ã_, edchat, musedchat, sxsw, sxswi, classical, newtwitter]",na,Neutral emotion,,unknown
53,"[mention, mention, on, the, locationbased, fast, fun, and, future, , link, via, mention, sxsw]",na,Neutral emotion,,unknown
66,"[at, sxsw, mention, , mention, wanna, buy, you, a, drink, 7pm, at, fado, on, 4th, link, join, us]",na,Neutral emotion,,unknown
71,"[chilcott, mention, sxsw, stand, talking, with, blogger, staff, too, late, to, win, competition, for, best, tweet, mentioning, mention, so, no, tshirt]",na,Neutral emotion,,unknown
...,...,...,...,...,...
8932,"[z6, no, news, is, good, news, link, codes, valid, 40075959p, 031111, infektd, sxsw, zlf]",na,Neutral emotion,,unknown
8936,"[client, news, mention, releases, dope, melodies, amp, heavy, bass, amp, invades, sxsw, gt, link]",na,Neutral emotion,,unknown
8970,"[this, is, my, 5th, year, downloading, the, sxsw, music, torrent, link, all, free, and, legal, great, music]",na,Neutral emotion,,unknown
9024,"[by, the, way, were, looking, for, a, spanishspeaking, trend, scout, based, in, austin, gt, link, sxsw]",na,Neutral emotion,,unknown


# Prepare for Testing

In [484]:
target_dict = {'Negative emotion': 2, 'Neutral emotion': 1, 'Positive emotion': 0}
df_test = df.copy()
df_test['target'] = df['is_there_an_emotion_directed_at_a_brand_or_product'].map(target_dict)
#df_test = df_test.loc[df_test['product'] != 'unknown']

In [485]:
df_test.drop(['emotion_in_tweet_is_directed_at','test','is_there_an_emotion_directed_at_a_brand_or_product'],axis=1,inplace=True)


In [486]:
X = df_test['tweet_text'].str.join(' ')
new_X = X.apply(tweets.tokenize)
y = df_test['target']

In [374]:
tweeted =tweeted.str.join(' ')

In [465]:
X_train,X_test,y_train,y_test = train_test_split(new_X.str.join(' '),y)

In [487]:
# Make Our test and train set
X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y)

In [488]:
# Our first model we will try a CountVectorizer with Naive Bayes
cv = CountVectorizer()

X_t_vec = cv.fit_transform(X_train)
X_t_vec = cv.fit_transform(X_train)
X_t_vec = pd.DataFrame.sparse.from_spmatrix(X_t_vec)
X_t_vec.columns = sorted(cv.vocabulary_)
X_t_vec.set_index(y_train.index, inplace=True)

In [489]:
X_val_vec = cv.transform(X_test)
X_val_vec  = pd.DataFrame.sparse.from_spmatrix(X_val_vec)
X_val_vec.columns = sorted(cv.vocabulary_)
X_val_vec.set_index(y_test.index, inplace=True)

In [490]:
mnb = MultinomialNB()

In [491]:
mnb.fit(X_t_vec,y_train)

MultinomialNB()

In [492]:
results = cross_validate(mnb,X_t_vec,y_train,return_train_score=True)

In [493]:
mean_scores(results)

Mean Train Score: 0.8307301743671898
Mean Test Score: 0.6653472695860219


From our first look at the model, it seems to be overfit. It has high variance from our test and train scores.

In [494]:
y_hat = cross_val_predict(mnb,X_t_vec,y_train)


In [495]:
metrics(y_train,y_hat)

Recall Score: 0.665346825047661
Precision Score: 0.665346825047661
Accuracy Score:0.665346825047661
F1 Score: 0.665346825047661
Confusion Matrix: 
[[1227  995   12]
 [ 832 3267   59]
 [ 165  219   43]]


In [496]:
lr_pipe = Pipeline([("vec" , CountVectorizer(stop_words='english')),
                       ("lr",LogisticRegression())])
results_LR = cross_validate(lr_pipe,X_train,y_train,return_train_score=True)
mean_scores(results_LR)

Mean Train Score: 0.9230092317575685
Mean Test Score: 0.6893942979842206


In [497]:
y_hat_train = cross_val_predict(lr_pipe, X_train, y_train)
metrics(y_train,y_hat_train)

Recall Score: 0.6893972723273207
Precision Score: 0.6893972723273207
Accuracy Score:0.6893972723273207
F1 Score: 0.6893972723273207
Confusion Matrix: 
[[1144 1071   19]
 [ 617 3483   58]
 [  87  266   74]]


In [498]:
pipe = make_pipeline(TfidfVectorizer(max_features=500), MultinomialNB())
cv = cross_validate(pipe, X_train, y_train,return_train_score=True)

In [499]:
mean_scores(cv)

Mean Train Score: 0.6849611389911056
Mean Test Score: 0.655814218678394


In [500]:
pipe = make_pipeline(TfidfVectorizer(), RandomForestClassifier())
cv = cross_validate(pipe, X_train, y_train,return_train_score=True)
mean_scores(cv)

Mean Train Score: 0.9967737164376087
Mean Test Score: 0.6737052560011876


Using a TFidfVectorizer, we have fixed our over fitting problem but the test score has not improved that much. Maybe we can tweak some hyperparameters to help improve the scores.

In [241]:
RF_pipe = Pipeline([( "tfid" , TfidfVectorizer(max_features=500)),
                       ("rf",RandomForestClassifier())])

params = {'rf__criterion':['gini', 'entropy'],
          'rf__max_depth':[None,3,5],
          'rf__min_samples_split':[2,5,10],
          'rf__min_samples_leaf':[1,3,5,7],
          'rf__max_features' : [5,10]}

clf = GridSearchCV(RF_pipe,params,return_train_score = True)


In [242]:
clf.fit(X_train,y_train)

GridSearchCV(estimator=Pipeline(steps=[('tfid',
                                        TfidfVectorizer(max_features=500)),
                                       ('rf', RandomForestClassifier())]),
             param_grid={'rf__criterion': ['gini', 'entropy'],
                         'rf__max_depth': [None, 3, 5],
                         'rf__max_features': [5, 10],
                         'rf__min_samples_leaf': [1, 3, 5, 7],
                         'rf__min_samples_split': [2, 5, 10]},
             return_train_score=True)

In [150]:
clf.best_params_

NameError: name 'clf' is not defined

In [244]:
gs_mean_scores(clf)

Mean Train Score: 0.6646375221793046'
Mean Test Score: 0.6230672320021744


In [501]:
lr_pipe = Pipeline([("tfid" , TfidfVectorizer(max_features=500,stop_words='english')),
                       ("lr",LogisticRegression())])
results_LR = cross_validate(lr_pipe,X_train,y_train,return_train_score=True)
mean_scores(results_LR)

Mean Train Score: 0.7158309253798742
Mean Test Score: 0.6695995765765959


In [502]:
lr_grid = {'lr__C' : [.5, 1, 50],
       'lr__max_iter' : [50,100,1750, 2000],
       }

clf_lr = GridSearchCV(lr_pipe,lr_grid,return_train_score=True)

In [ ]:
LR_results = clf_LR.fit(X_train,y_train)
gs_m

In [504]:
lr_results = clf_lr.fit(X_train,y_train)
gs_mean_scores(lr_results)

Mean Train Score: 0.7188860919296151'
Mean Test Score: 0.663917534275852


In [505]:
lr_results.best_params_

{'lr__C': 0.5, 'lr__max_iter': 100}

In [506]:
common_words = ['sxsw', 'mention', 'link', 'ipad', 'iphone', 'google', 'apple', '2', 'android', 'rt']

In [507]:
sgd_pipe = Pipeline([("tfid" , TfidfVectorizer(stop_words=common_words + sw.words('english'))),
                       ("sgd",SGDClassifier())])
results_sgd = cross_validate(sgd_pipe,X_train,y_train,return_train_score=True)
mean_scores(results_sgd)

Mean Train Score: 0.9071344726649123
Mean Test Score: 0.6808911900822535


In [508]:
results_sgd = clf_sgd.fit(X_train,y_train)

In [509]:
y_pred = cross_val_predict(sgd_pipe,X_train,y_train)
metrics(y_train,y_pred)

Recall Score: 0.6811849244757295
Precision Score: 0.6811849244757295
Accuracy Score:0.6811849244757295
F1 Score: 0.6811849244757295
Confusion Matrix: 
[[1143 1071   20]
 [ 686 3416   56]
 [  91  250   86]]


In [510]:
gs_mean_scores(results_sgd)

Mean Train Score: 0.7479162500347192'
Mean Test Score: 0.6479433592307234


In [517]:
y_hat_train = cross_val_predict(sgd_pipe, X_train, y_train)
metrics(y_train,y_hat_train)

Recall Score: 0.6800117319255022
Precision Score: 0.6800117319255022
Accuracy Score:0.6800117319255022
F1 Score: 0.6800117319255022
Confusion Matrix: 
[[1131 1083   20]
 [ 682 3421   55]
 [  92  250   85]]


In [512]:
xgb_pipe = Pipeline([("tfid" , TfidfVectorizer()),
                       ("xgb",XGBClassifier())])
results_xgb = cross_validate(xgb_pipe,X_train,y_train,return_train_score=True)
mean_scores(results_xgb)

Mean Train Score: 0.8929828562673541
Mean Test Score: 0.6776643078597969


In [520]:
xgb_pipe = Pipeline([("count" , CountVectorizer()),
                       ("xgb",XGBClassifier())])
results_xgb = cross_validate(xgb_pipe,X_train,y_train,return_train_score=True)
mean_scores(results_xgb)

Mean Train Score: 0.8431220650967258
Mean Test Score: 0.6914468687538056


In [514]:
xgb_grid = {'xgb__n_estimators':[50,75,100],
            'xgb__max_depth':[1,3,4,5]}

xgb_gs = GridSearchCV(estimator=xgb_pipe, param_grid=xgb_grid, return_train_score=True)

In [521]:
results_xgb = xgb_gs.fit(X_train,y_train)

In [522]:
gs_mean_scores(results_xgb)

Mean Train Score: 0.744308088013275'
Mean Test Score: 0.673068202437123


In [525]:
xgb_pred = cross_val_predict(xgb_gs.best_estimator_, X_train, y_train)

In [526]:
metrics(y_train,xgb_pred)

Recall Score: 0.6870508872268661
Precision Score: 0.6870508872268661
Accuracy Score:0.6870508872268661
F1 Score: 0.6870508872268661
Confusion Matrix: 
[[ 877 1340   17]
 [ 394 3733   31]
 [  61  291   75]]


In [ ]:
XGBClassifier()